In [14]:
#process large files and write there data to files that can be read by other code.
import numpy as np
import yt
import time
from yt.funcs import mylog
import matplotlib.pyplot as plt
mylog.setLevel(40) #prevents warning

In [21]:
#setup
#this is the only cell to be cehanged for each new image. npz.save seems to not be able to 
# handle to many of theseinputs at once. 4 seems to work best at one time 

inputfile  = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0373_g10", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0127_gt10", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0309_gtm10", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0282_gtmp6")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0361_g5", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0105_gt5", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0276_gtm5", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0255_gtmp5")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0356_g4", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0100_gt4", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0272_gtm4", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0225_gtmp4")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0347_g3", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0096_gt3", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0252_gtm3", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0201_gtmp3")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0338_g2", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0092_gt2", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0232_gtm2","/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0178_gtmp2")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0330_g1", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0087_gt1", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0187_gtm1", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0132_gtmp1")
#inputfile = ("/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0321_g0", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0077_gt0", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0116_gtm0", "/scratch/jak491/hdf5_files/OUT_hdf5_plt_cnt_0077_gtmp0" )
outputfile = ('/scratch/jak491/npz_storage/vol_covering_grid_den_high')
variable_name = ('density_g','density_gt','density_gtm', 'density_gtmp')

In [22]:
#load in file and make covering grid
start = time.time()
print('starting')
d = {}
for a in range(4):
    ds = yt.load(inputfile[a])
    level2 = ds.covering_grid(2,left_edge=ds.domain_left_edge,dims=ds.domain_dimensions*2**2)
    d[variable_name[a]] = level2['density']
    
end = time.time()
print("time:",(end-start)/60)

starting
time: 2.9010133266448976


In [23]:
#load in file and make covering grid
start = time.time()
print('starting')
#process complex pieces
b=0
full_bins = [0,0,0,0]
full_counts = [0,0,0,0]
axis = ['x','y','z']
for n in range(4):
    print(b)
    ds = yt.load(inputfile[n])
    all_data = ds.all_data()
    #these lines call specific variables from covering grid
    den = all_data['density']
    cv = all_data['cell_volume']
    ave_vol = np.mean(cv)
    den_w= den*all_data['cell_volume']/ave_vol#den.flatten()
    den_mean = np.mean(den_w)
    log_den = np.log(den/den_mean)
    #make the actual histogram and determine how many bins to split into
    counts_d, binS_d = np.histogram(log_den,bins=50, weights = cv,density = True)
    #bin_list and counts_d are what is actually plotted
    bin_list = [] 
    for i in range(len(binS_d)-1):
        bin_i=(binS_d[i]+binS_d[i+1])/2
        bin_list.append(bin_i)
    full_bins[n] = bin_list
    full_counts[n] = counts_d
    #next parts where originally in plotting notebooks
    b = b+1
np.savez(outputfile,  bins_g = full_bins[0],bins_gt = full_bins[1], bins_gtm = full_bins[2], bins_gtmp = full_bins[3], counts_g = full_counts[0], counts_gt = full_counts[1], counts_gtm = full_counts[2], counts_gtmp = full_counts[3])       
#np.savez(outputfile, density_g = density_g, density_gt = density_gt , density_gtm = density_gtm, density_gtmp = density_gtmp, log_vol_g = log_vol[0],log_vol_gt = log_vol[1], log_vol_gtm = log_vol[2], log_vol_gtmp = log_vol[3])
end = time.time()
print("time:",(end-start)/60)

starting
0
1
2
3
time: 3.776434671878815
